In [ ]:
%load_ext autoreload
%autoreload 2

# Import Required Libraries
Import the necessary pdf2image and OpenAI.

In [ ]:
# Importing required libraries
from fastcore.all import AttrDict
import io
from pdf2image import convert_from_path
from pypdf import PdfReader
import openai
import os
import base64
import requests
from dotenv import load_dotenv
import os

load_dotenv()  # Carga las variables de entorno desde .env

# Load and Convert PDF to Image
Use PyPDF2 pdf2image to load the PDF file and convert it into an image.

In [ ]:
# Load the PDF file
path = '../example_data/1.pdf'

# Open and read the PDF file
reader = PdfReader(path)

# Check if the PDF has more than one page
if len(reader.pages) > 1:
    print("The PDF has more than one page. Only the first page will be converted to an image.")

# Convert the first page of the PDF to an image
images = convert_from_path(path, first_page=1, last_page=1, fmt='jpeg')


In [ ]:
# Crop the image
image = images[0].crop((0, 0, images[0].width, 450))
image

# Call OpenAI ChatGPT API with Vision Model
Use the OpenAI API to call the ChatGPT model with the vision capability.

In [ ]:
buffered = io.BytesIO()
images[0].save(buffered, format="JPEG")
base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
base64_image

In [ ]:
# Load moodle students information
import pandas as pd

# Cargar el CSV de alumnos y grupos
students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
dfs = [ pd.read_csv(filename) for filename in students_info ]

df = pd.concat(dfs, ignore_index=True)

# Puedes opcionalmente limpiar espacios y convertir a mayúsculas para facilitar coincidencias
df["Nombre"] = df["Nombre"].str.strip().str.upper()
df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()

guia_texto = "\n".join(
    f"{row['Nombre']} {row['Apellido(s)']} - Grupo: {row['Grupos']}"
    for _, row in df.iterrows()
)

In [ ]:
#print(guia_texto[:3])
#print(guia_texto[3:])

In [ ]:
api_key = os.environ["OPENAI_API_KEY"] # *
prompt = """Extract the last name (Apellidos in Spanish), the first name (Nombre in Spanish) and the group (Grupo in Spanish)
 from the top of the image. You will find them handwritten after the labels `Apellidos`,  `Nombre` and `Grupo` respectively. The fields of your JSON output will have those exact same label names
 Here is a list of expected students and their groups as a reference: 
 {guia_texto}
 """ # *
model = "gpt-4o"

In [ ]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": f'{model}',
  "response_format": { "type": "json_object" },
  "messages": [
    {
        "role": "system", 
        "content": "You are a helpful assistant designed to see an exam and output JSON \
           with the extracted information. You will be given an image of the exam.\
           the default group in case of empty string is extraviado",   
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{prompt}"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [ ]:
import json

# Obtener el JSON como string
info_str = response.json()['choices'][0]['message']['content']

# Convertir a diccionario Python
info = json.loads(info_str)

# Si el grupo está vacío, marcarlo como "extraviado"
if not info.get("Grupo"):  # También cubre None y ""
    info["Grupo"] = "extraviado"

# Mostrar resultado
print(info)


In [ ]:
# Add path to the info dictionary
info['path'] = path

In [ ]:
path

In [ ]:
import pandas as pd
import os
import zipfile
from pathlib import Path
import unicodedata
import re

def normalizar_texto(texto):
    """Normaliza texto eliminando acentos y caracteres especiales"""
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(char for char in texto if unicodedata.category(char) != 'Mn')
    texto = texto.upper().strip()
    texto = re.sub(r'[^A-Z0-9\s]', '', texto)
    return texto

def buscar_practica_en_zips(apellidos, nombre, practica_num=3, ruta_data="./../data/"):
    """
    Busca si existe una práctica para un alumno en los archivos ZIP
    
    Args:
        apellidos: Apellidos del alumno
        nombre: Nombre del alumno
        practica_num: Número de práctica (3 o 5)
        ruta_data: Ruta base a la carpeta data
    """
    ruta_practica = Path(ruta_data) / f"Practica{practica_num}"
    
    if not ruta_practica.exists():
        return False
    
    # Normalizar apellidos y nombre
    apellidos_norm = normalizar_texto(apellidos)
    nombre_norm = normalizar_texto(nombre)
    
    # Buscar en todos los archivos ZIP
    for archivo_zip in ruta_practica.glob("*.zip"):
        try:
            with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
                for archivo in zip_ref.namelist():
                    archivo_norm = normalizar_texto(archivo)
                    
                    # Verificar si el archivo contiene apellidos y nombre
                    if apellidos_norm in archivo_norm and nombre_norm in archivo_norm:
                        return True
        except Exception as e:
            continue
    
    return False

def verificar_todas_las_practicas(df, ruta_data="./../data/"):
    """
    Añade columnas de verificación de prácticas al DataFrame existente
    
    Args:
        df: DataFrame con columnas 'Nombre' y 'Apellido(s)'
        ruta_data: Ruta base a la carpeta data
    
    Returns:
        DataFrame modificado con las nuevas columnas
    """
    
    # Crear copias para evitar warnings
    df_resultado = df.copy()
    
    # Inicializar las nuevas columnas
    df_resultado['Presentada_3'] = 0
    df_resultado['Comentario_3'] = 'NP'
    df_resultado['Presentada_5'] = 0
    df_resultado['Comentario_5'] = 'NP'
    
    print("Verificando entregas de prácticas...")
    print("="*50)
    
    practicas_3_encontradas = 0
    practicas_5_encontradas = 0
    total_alumnos = len(df_resultado)
    
    for idx, row in df_resultado.iterrows():
        nombre = str(row['Nombre'])
        apellidos = str(row['Apellido(s)'])
        
        # Verificar Práctica 3
        tiene_practica3 = buscar_practica_en_zips(apellidos, nombre, 3, ruta_data)
        if tiene_practica3:
            df_resultado.loc[idx, 'Presentada_3'] = 1
            df_resultado.loc[idx, 'Comentario_3'] = ''
            practicas_3_encontradas += 1
        
        # Verificar Práctica 5
        tiene_practica5 = buscar_practica_en_zips(apellidos, nombre, 5, ruta_data)
        if tiene_practica5:
            df_resultado.loc[idx, 'Presentada_5'] = 1
            df_resultado.loc[idx, 'Comentario_5'] = ''
            practicas_5_encontradas += 1
        
        # Mostrar progreso
        status_3 = "✓" if tiene_practica3 else "✗"
        status_5 = "✓" if tiene_practica5 else "✗"
        print(f"{status_3} P3 | {status_5} P5 | {apellidos}, {nombre}")
    
    print("="*50)
    print(f"RESUMEN:")
    print(f"Total alumnos verificados: {total_alumnos}")
    print(f"Práctica 3 - Entregadas: {practicas_3_encontradas} | No entregadas: {total_alumnos - practicas_3_encontradas}")
    print(f"Práctica 5 - Entregadas: {practicas_5_encontradas} | No entregadas: {total_alumnos - practicas_5_encontradas}")
    
    return df_resultado

In [15]:
# Load moodle students information
import pandas as pd
# Cargar el CSV de alumnos y grupos
students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
dfs = [ pd.read_csv(filename) for filename in students_info ]
df = pd.concat(dfs, ignore_index=True)
# Limpiar espacios y convertir a mayúsculas
df["Nombre"] = df["Nombre"].str.strip().str.upper()
df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
print("DataFrame original cargado:")
print(f"Total alumnos: {len(df)}")
print(df.head())
print("\n")

# Verificar prácticas y añadir columnas
df_con_practicas = verificar_todas_las_practicas(df)

print("\nDataFrame con verificación de prácticas:")
print(df_con_practicas.head())

# Mostrar estadísticas por grupo si existe la columna
if 'Grupos' in df_con_practicas.columns:
    print("\nEstadísticas por grupo:")
    resumen_grupos = df_con_practicas.groupby('Grupos').agg({
        'Presentada_3': 'sum',
        'Presentada_5': 'sum'
    })
    print(resumen_grupos)

✗ P3 | ✓ P5 | JIMENEZ JIMENEZ, DIEGO
✗ P3 | ✓ P5 | JIMENEZ RAMOS, DANIEL
✗ P3 | ✓ P5 | JUAREZ GELARDO, TOMAS
✗ P3 | ✓ P5 | JUSUE ZAVALA, JOSE RAMON
✗ P3 | ✓ P5 | KE, TAILI
✗ P3 | ✗ P5 | LABRADA MEDINA, JAVIER
✗ P3 | ✓ P5 | LAFUENTE SANZ, ALICIA
✗ P3 | ✓ P5 | LEFTERACHE RAILEANU, NICOLAS ANDRES
✗ P3 | ✓ P5 | LENCERO CARRILLO, OSCAR
✗ P3 | ✓ P5 | LI, JILING
✓ P3 | ✗ P5 | LIN, CRISTIAN
✗ P3 | ✗ P5 | LIN, YUSHAN
✗ P3 | ✓ P5 | LLORENTE VAQUERO, CARLOS
✗ P3 | ✓ P5 | LOPEZ COLMENERO, ROSALIA
✗ P3 | ✓ P5 | LOPEZ DE LA MANZANARA GARCIA, PABLO
✗ P3 | ✓ P5 | LOPEZ HERNANDEZ, ANDRES
✗ P3 | ✗ P5 | LOPEZ SOSA, JORGE
✗ P3 | ✓ P5 | LORENZO MORO, ADRIAN
✗ P3 | ✓ P5 | LOZANO MARCOS, MARTA
✗ P3 | ✗ P5 | LU DONG, LUIS
✗ P3 | ✓ P5 | MA, ANNI
✗ P3 | ✗ P5 | MADRIDEJOS CHAMORRO, TELLO
✗ P3 | ✓ P5 | MAHER FAIQ AL RAWE, MAHMOOD
✓ P3 | ✓ P5 | MANZANARO CARABALLO, PABLO
✗ P3 | ✓ P5 | MARINA NAVARRO, PAULA
✗ P3 | ✓ P5 | MARQUEZ SANTAMARIA, ALVARO
✗ P3 | ✓ P5 | MARTIN BALLESTER, DANIEL
✗ P3 | ✓ P5 | MARTIN ESPAÑA, 

In [16]:
display(df_con_practicas)

,Nombre,Apellido(s),Dirección de correo,Grupos,Presentada_3,Comentario_3,Presentada_5,Comentario_5
0,SOFIA,AGAPITO DELGADO,s.agapito@alumnos.upm.es,IWSIT11,0,NP,1,
1,LLOYD DAREN,AGUILAR DESIAR,daren.aguilar@alumnos.upm.es,IWSIM12,0,NP,1,
2,JAVIER,AGUIRRE HERVIAS,javier.aguirrehervias@alumnos.upm.es,IWSIM12,0,NP,1,
3,MATEO,ALBRIZIO,mateo.albrizio@alumnos.upm.es,IWSIM12,0,NP,1,
4,NICOLAS,ALONSO FERNANDEZ,ni.alonso@alumnos.upm.es,IWSIM12,0,NP,0,NP
...,...,...,...,...,...,...,...,...
439,HAOQING,ZHANG,haoqing.zhang@alumnos.upm.es,CITIT11,0,NP,0,NP
440,JIONGHAO,ZHANG,jionghao.zhang@alumnos.upm.es,CITIM12,1,,1,
441,STEVEN WEI,ZHANG XIA,steven.zhang@alumnos.upm.es,CITIT11,0,NP,1,
442,YI,ZHOU,yi.zhou@alumnos.upm.es,CITIT11,0,NP,1,


In [17]:
df_con_practicas.to_csv("../data/practicas_3_5")

In [18]:
# Seleccionar un grupo en concreto
columnas_importantes = ['Nombre', 'Apellido(s)', 'Presentada_5', 'Comentario_5']
df_con_practicas[df_con_practicas['Grupos'] == 'CITIT11'][columnas_importantes].to_csv('../data/alumnos_CITIT11.csv', index=False)

In [19]:
def renombrar_archivos_en_lotes(ruta="../data/raw/"):
	"""
	Renombra todos los archivos en la carpeta dada como 'lote_1', 'lote_2', etc.
	Conserva la extensión original de cada archivo.
	"""
	archivos = sorted([f for f in os.listdir(ruta) if os.path.isfile(os.path.join(ruta, f))])
	for idx, nombre_original in enumerate(archivos, start=1):
		extension = os.path.splitext(nombre_original)[1]
		nuevo_nombre = f"lote_{idx}{extension}"
		ruta_origen = os.path.join(ruta, nombre_original)
		ruta_destino = os.path.join(ruta, nuevo_nombre)
		os.rename(ruta_origen, ruta_destino)
	print(f"Renombrados {len(archivos)} archivos en '{ruta}'.")
#renombrar_archivos_en_lotes()

In [20]:
import os
from pathlib import Path
from pypdf import PdfReader, PdfWriter

def crear_carpeta_examenes(base_dir="../data", nombre_base="examenes"):
    """
    Crea una carpeta nueva para los exámenes. Si ya existe, añade un sufijo numérico.
    """
    base_path = Path(base_dir)
    carpeta = base_path / nombre_base
    contador = 1
    while carpeta.exists():
        carpeta = base_path / f"{nombre_base}_{contador}"
        contador += 1
    carpeta.mkdir(parents=True)
    return carpeta

def dividir_pdf_en_examenes(pdf_path, carpeta_destino, nombre_base="examen"):
    """
    Divide un PDF en archivos de 2 páginas cada uno y los guarda en la carpeta destino.
    """
    reader = PdfReader(pdf_path)
    num_paginas = len(reader.pages)
    examen_idx = 1
    for i in range(0, num_paginas, 2):
        writer = PdfWriter()
        writer.add_page(reader.pages[i])
        if i+1 < num_paginas:
            writer.add_page(reader.pages[i+1])
        nombre_examen = f"{nombre_base}_{examen_idx}.pdf"
        ruta_examen = carpeta_destino / nombre_examen
        with open(ruta_examen, "wb") as f_out:
            writer.write(f_out)
        examen_idx += 1

def procesar_lotes_y_generar_examenes(ruta_lotes="../data/raw/", base_dir="../data", nombre_carpeta="examenes"):
    """
    Busca todos los archivos PDF en la carpeta de lotes, los divide de 2 en 2 páginas y los guarda en una carpeta nueva.
    """
    carpeta_destino = crear_carpeta_examenes(base_dir, nombre_carpeta)
    archivos_lote = sorted([f for f in os.listdir(ruta_lotes) if f.lower().endswith(".pdf")])
    examen_global_idx = 1
    for archivo in archivos_lote:
        ruta_pdf = Path(ruta_lotes) / archivo
        reader = PdfReader(ruta_pdf)
        num_paginas = len(reader.pages)
        for i in range(0, num_paginas, 2):
            writer = PdfWriter()
            writer.add_page(reader.pages[i])
            if i+1 < num_paginas:
                writer.add_page(reader.pages[i+1])
            nombre_examen = f"examen_{examen_global_idx}.pdf"
            ruta_examen = carpeta_destino / nombre_examen
            with open(ruta_examen, "wb") as f_out:
                writer.write(f_out)
            examen_global_idx += 1
    print(f"Exámenes generados en: {carpeta_destino}")

# Ejemplo de uso:
procesar_lotes_y_generar_examenes()

Exámenes generados en: ..\data\examenes


In [21]:
import os
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader, PdfWriter
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Dropdown
from IPython.display import display, clear_output

class JupyterPDFReviewer:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.reader = PdfReader(pdf_path)
        self.total_pages = len(self.reader.pages)
        self.current_index = 0
        self.images = {}
        self.output_dir = Path("../data/saved")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.out = Output()
        self.status = Label(value="")  # Estado visual
        self._setup_widgets()
        self._show_pages()

    def _setup_widgets(self):
        self.btn_save1 = Button(description='Save Page 1', layout=Layout(width='120px'))
        self.btn_save12 = Button(description='Save Pages 1&2', layout=Layout(width='120px'))
        self.btn_save123 = Button(description='Save Pages 1-3', layout=Layout(width='120px'))
        self.btn_next = Button(description='Next (Skip 1)', layout=Layout(width='120px'))
        self.btn_prev = Button(description='Previous', layout=Layout(width='120px'))

        self.btn_save1.on_click(lambda x: self._save_pages([0]))
        self.btn_save12.on_click(lambda x: self._save_pages([0, 1]))
        self.btn_save123.on_click(lambda x: self._save_pages([0, 1, 2]))
        self.btn_next.on_click(lambda x: self._next_page())
        self.btn_prev.on_click(lambda x: self._prev_page())

        display(VBox([
            HBox([self.btn_prev, self.btn_save1, self.btn_save12, self.btn_save123, self.btn_next]),
            self.status,
            self.out
        ]))

    def _get_page_image(self, idx):
        if idx not in self.images and idx < self.total_pages:
            self.status.value = f"Cargando página {idx+1}..."
            try:
                img = convert_from_path(
                    self.pdf_path,
                    first_page=idx + 1,
                    last_page=idx + 1,
                    dpi=50,
                    fmt='jpeg',
                    thread_count=1
                )[0]
                self.images[idx] = img
            except Exception as e:
                self.status.value = f"Error cargando página {idx+1}"
                print(f"Error converting page {idx+1}: {e}")
                return None
        self.status.value = ""
        return self.images.get(idx)

    def _show_pages(self):
        with self.out:
            clear_output(wait=True)
            fig, axes = plt.subplots(1, 3, figsize=(15, 8))
            for i in range(3):
                page_idx = self.current_index + i
                axes[i].axis('off')
                if page_idx < self.total_pages:
                    img = self._get_page_image(page_idx)
                    if img is not None:
                        axes[i].imshow(img)
                        axes[i].set_title(f"Page {page_idx+1}")
                    else:
                        axes[i].set_title(f"Page {page_idx+1} (error)")
                else:
                    axes[i].set_title("No Page")
            plt.show()

    def _next_page(self):
        if self.current_index + 1 < self.total_pages:
            self.current_index += 1
            self._show_pages()

    def _prev_page(self):
        if self.current_index >= 1:
            self.current_index -= 1
            self._show_pages()

    def _save_pages(self, rel_indices):
        abs_indices = [self.current_index + i for i in rel_indices if self.current_index + i < self.total_pages]
        if not abs_indices:
            self.status.value = "No valid pages to save"
            return

        base_name = Path(self.pdf_path).stem  # Ejemplo: 'lote_1'
        lote = base_name
        examen_n = abs_indices[0] + 1  # Primer índice de página + 1
        output_path = self.output_dir / f"{lote}_examen_{examen_n}.pdf"

        writer = PdfWriter()
        for idx in abs_indices:
            writer.add_page(self.reader.pages[idx])
        with open(output_path, "wb") as f:
            writer.write(f)
        self.status.value = f"Guardado: {output_path.name}"

        # Avanzar tantas páginas como se han guardado
        avance = len(abs_indices)
        if self.current_index + avance < self.total_pages:
            self.current_index += avance
            self._show_pages()

def revisar_todos_los_lotes(ruta_lotes="../data/raw/"):
    archivos = sorted([f for f in os.listdir(ruta_lotes) if f.lower().endswith(".pdf")])
    if not archivos:
        print("No se encontraron lotes PDF en la carpeta.")
        return
    dropdown = Dropdown(options=archivos, description='Lote:', layout=Layout(width='50%'))
    out = Output()

    def on_select(change):
        with out:
            clear_output(wait=True)
            print(f"Revisando: {dropdown.value}")
            JupyterPDFReviewer(os.path.join(ruta_lotes, dropdown.value))

    dropdown.observe(on_select, names='value')
    display(VBox([dropdown, out]))
    # Mostrar el primero por defecto
    on_select({'new': archivos[0]})

# Ejemplo de uso:
# revisar_todos_los_lotes("../data/raw/")

In [22]:
#revisar_todos_los_lotes("../data/raw/") # --> solo cuando necesites procesar los lotes

In [23]:
import os
import json
import base64
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
import pandas as pd
import shutil
import io
import requests

def procesar_examenes_completo(carpeta_examenes="../data/saved/", output_dir="../data/examenes_procesados/"):
    """
    Procesa todos los exámenes de la carpeta, extrae información con OpenAI,
    los organiza por grupo y práctica, y genera un DataFrame de seguimiento.
    """
    # Crear directorio de salida
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Lista para almacenar información de exámenes procesados
    examenes_info = []
    
    # Obtener todos los archivos PDF
    pdf_files = list(Path(carpeta_examenes).glob("*.pdf"))
    
    print(f"Procesando {len(pdf_files)} exámenes...")
    
    # Variables necesarias para la API
    api_key = os.environ["OPENAI_API_KEY"]
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    for pdf_file in pdf_files:
        print(f"\nProcesando: {pdf_file.name}")
        
        try:
            # Procesar cada página del PDF
            reader = PdfReader(pdf_file)
            num_pages = len(reader.pages)
            
            for page_num in range(min(2, num_pages)):  # Solo primeras 2 páginas
                # Convertir página a imagen
                images = convert_from_path(
                    pdf_file, 
                    first_page=page_num + 1, 
                    last_page=page_num + 1, 
                    dpi=150,
                    fmt='jpeg'
                )
                
                if not images:
                    continue
                    
                # Convertir imagen a base64
                buffered = io.BytesIO()
                images[0].save(buffered, format="JPEG")
                base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
                
                # Llamar a OpenAI
                info = extraer_info_con_openai(base64_image, headers)
                
                if info and info.get('Apellidos') and info.get('Nombre'):
                    info['archivo_original'] = pdf_file.name
                    info['pagina'] = page_num + 1
                    
                    # Organizar archivo por grupo y práctica
                    mover_archivo_organizado(pdf_file, info, output_path)
                    
                    examenes_info.append(info)
                    practica = info.get('practica_detectada', 'desconocida')
                    print(f"✓ Extraído: {info['Nombre']} {info['Apellidos']} - Grupo: {info['Grupo']} - Práctica: {practica}")
                    break  # Si encontramos info en una página, no procesar más páginas
                    
        except Exception as e:
            print(f"✗ Error procesando {pdf_file.name}: {e}")
            continue
    
    # Crear DataFrame de seguimiento
    df_examenes = crear_dataframe_examenes(examenes_info)
    
    # Guardar DataFrame
    df_examenes.to_csv(output_path / "seguimiento_examenes.csv", index=False)
    
    print(f"\n📊 Procesamiento completado:")
    print(f"- Exámenes procesados: {len(examenes_info)}")
    print(f"- Archivo de seguimiento guardado en: {output_path / 'seguimiento_examenes.csv'}")
    
    return df_examenes

def extraer_info_con_openai(base64_image, headers):
    """Extrae información del examen usando OpenAI"""
    try:
        prompt_completo = f"""Extract the last name (Apellidos in Spanish), the first name (Nombre in Spanish) and the group (Grupo in Spanish)
         from the top of the image. You will find them handwritten after the labels `Apellidos`,  `Nombre` and `Grupo` respectively. 
         The fields of your JSON output will have those exact same label names.
         Also determine if this is a "Práctica de Listas" (practice 3) or "Práctica de Grafos" (practice 5) based on the content.
         Add a field called "practica_detectada" with value 3 for listas or 5 for grafos.
         Here is a list of expected students and their groups as a reference: 
         {guia_texto}
         """
        
        payload = {
            "model": "gpt-4o",
            "response_format": {"type": "json_object"},
            "messages": [
                {
                    "role": "system", 
                    "content": "You are a helpful assistant designed to see an exam and output JSON with the extracted information. The default group in case of empty string is extraviado",   
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_completo
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        
        response = requests.post(
            "https://api.openai.com/v1/chat/completions", 
            headers=headers, 
            json=payload
        )
        
        if response.status_code == 200:
            info_str = response.json()['choices'][0]['message']['content']
            info = json.loads(info_str)
            
            # Si el grupo está vacío, marcarlo como "extraviado"
            if not info.get("Grupo"):
                info["Grupo"] = "extraviado"
                
            return info
        else:
            print(f"Error en API OpenAI: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error extrayendo información: {e}")
        return None

def mover_archivo_organizado(archivo_original, info, output_path):
    """Organiza el archivo en carpetas por GRUPO y luego por PRÁCTICA"""
    try:
        practica = info.get('practica_detectada', 'desconocida')
        grupo = info.get('Grupo', 'extraviado')
        
        # Crear estructura de carpetas: GRUPO -> PRÁCTICA
        carpeta_grupo = output_path / grupo
        carpeta_practica = carpeta_grupo / f"Practica_{practica}"
        carpeta_practica.mkdir(parents=True, exist_ok=True)
        
        # Generar nombre de archivo
        apellidos = info.get('Apellidos', 'SinApellidos').replace(' ', '_')
        nombre = info.get('Nombre', 'SinNombre').replace(' ', '_')
        
        nombre_base = f"{apellidos}_{nombre}"
        extension = archivo_original.suffix
        nuevo_archivo = carpeta_practica / f"{nombre_base}{extension}"
        
        # Si ya existe, añadir sufijo numérico
        contador = 2
        while nuevo_archivo.exists():
            nuevo_archivo = carpeta_practica / f"{nombre_base}_{contador}{extension}"
            contador += 1
        
        # Copiar archivo
        shutil.copy2(archivo_original, nuevo_archivo)
        print(f"  → Guardado en: {carpeta_grupo.name}/{carpeta_practica.name}/{nuevo_archivo.name}")
        
    except Exception as e:
        print(f"Error organizando archivo {archivo_original.name}: {e}")

def crear_dataframe_examenes(examenes_info):
    """Crea DataFrame de seguimiento de exámenes"""
    # Crear DataFrame base con todos los estudiantes
    df_base = df[['Nombre', 'Apellido(s)', 'Grupos']].copy()
    
    # Inicializar columnas de exámenes
    df_base['Examen_3'] = 0
    df_base['Comentario_Examen_3'] = 'PNP'  # PNP = No Presentado
    df_base['Examen_5'] = 0
    df_base['Comentario_Examen_5'] = 'PNP'  # PNP = No Presentado
    
    # Procesar información de exámenes
    for examen in examenes_info:
        practica = examen.get('practica_detectada')
        if practica in [3, 5, '3', '5']:
            practica = str(practica)
            
            # Buscar estudiante en el DataFrame
            apellidos = examen.get('Apellidos', '').upper().strip()
            nombre = examen.get('Nombre', '').upper().strip()
            
            # Buscar coincidencia (más flexible)
            mask = df_base['Apellido(s)'].str.upper().str.contains(apellidos[:5] if len(apellidos) > 5 else apellidos, na=False, regex=False) & \
                   df_base['Nombre'].str.upper().str.contains(nombre[:5] if len(nombre) > 5 else nombre, na=False, regex=False)
            
            if mask.any():
                df_base.loc[mask, f'Examen_{practica}'] = 1
                df_base.loc[mask, f'Comentario_Examen_{practica}'] = ''  # Limpiar PNP
                print(f"  → Marcado como presentado: {nombre} {apellidos} - Práctica {practica}")
    
    return df_base



In [24]:
import os
import json
import base64
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
import pandas as pd
import shutil
import io
import requests
from rapidfuzz import fuzz
import re
import unicodedata

def limpiar_texto(texto):
    """Elimina acentos, símbolos y deja solo letras/números/espacios en mayúsculas"""
    if not texto:
        return ""
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')
    texto = texto.upper()
    texto = re.sub(r'[^A-Z0-9\s]', '', texto)
    texto = texto.strip()
    return texto

def buscar_grupo_flexible(nombre, apellidos, df, texto_ocr=None, umbral=90):
    """
    Busca el grupo del alumno en el DataFrame por nombre y apellidos usando fuzzy.
    Si no encuentra por el umbral, devuelve el grupo más parecido aunque el score sea bajo (mínimo 20%).
    """
    nombre = limpiar_texto(nombre)
    apellidos = limpiar_texto(apellidos)
    mejor_score = -1
    mejor_grupo = None
    mejor_nombre = ""
    mejor_apellidos = ""
    for _, row in df.iterrows():
        nombre_df = limpiar_texto(str(row["Nombre"]))
        apellidos_df = limpiar_texto(str(row["Apellido(s)"]))
        score_nombre = fuzz.ratio(nombre, nombre_df)
        score_apellidos = fuzz.ratio(apellidos, apellidos_df)
        score = (score_nombre + score_apellidos) / 2
        if score > mejor_score:
            mejor_score = score
            mejor_grupo = row["Grupos"]
            mejor_nombre = row["Nombre"]
            mejor_apellidos = row["Apellido(s)"]
    # Si supera el umbral, devuelve el grupo y nombre exactos
    if mejor_score >= umbral:
        return mejor_grupo, mejor_nombre, mejor_apellidos
    # Si no supera el umbral pero hay algún match > 20%, devuelve el más parecido
    if mejor_score >= 20:
        return mejor_grupo, mejor_nombre, mejor_apellidos
    # Si no hay nada ni con 20%, busca patrón OCR
    if texto_ocr:
        texto_ocr = texto_ocr.upper()
        patrones = [
            r"CITI[TM][1][12]", r"IWSI[TM][1][12]", r"IWSIT[1][12]", r"CITIT[1][12]", r"IWSIM[1][12]"
        ]
        for patron in patrones:
            match = re.search(patron, texto_ocr)
            if match:
                return match.group(0), "", ""
    return "extraviado", "", ""

def extraer_info_con_openai(base64_image, headers, df):
    try:
        prompt_completo = (
            "Extract the last name (Apellidos in Spanish), "
            "the first name (Nombre in Spanish) from the top of the image. "
            "Extract also the group (Grupo in Spanish) from the top of the image. "
            "You will find them handwritten after the labels `Apellidos` and `Nombre` and `Group` respectively. "
            "The fields of your JSON output will have those exact same label names. "
            "Also determine if this is a \"Práctica de Listas\" (practice 3) or \"Práctica de Grafos\" (practice 5) based on the content. "
            "Add a field called \"practica_detectada\" with value 3 for listas or 5 for grafos. "
            "If you see a group label like CITIM11, CITIM12, IWSIM11, IWSIM12, CITIT11, IWSIT11, IWSIT12, include it as the field 'Grupo'."
        )
        payload = {
            "model": "gpt-4o",
            "response_format": {"type": "json_object"},
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant designed to see an exam and output JSON with the extracted information."
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_completo},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                    ]
                }
            ],
            "max_tokens": 300
        }
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload,
                timeout=10
            )
        except requests.exceptions.Timeout:
            return None, "Timeout: La petición a OpenAI tardó más de 10 segundos."
        except requests.exceptions.RequestException as e:
            return None, f"Error de conexión con OpenAI: {e}"

        if response.status_code == 200:
            info_str = response.json()['choices'][0]['message']['content']
            try:
                info = json.loads(info_str)
                nombre_ocr = info.get("Nombre", "")
                apellidos_ocr = info.get("Apellidos", "")
                grupo_ocr = info.get("Grupo", "")
                grupo, nombre, apellidos = buscar_grupo_flexible(nombre_ocr, apellidos_ocr, df, texto_ocr=grupo_ocr)
                info["Grupo"] = grupo
                # Si se encontró en el excel, usa los nombres del excel (más limpios)
                if grupo != "extraviado" and nombre and apellidos:
                    info["Nombre"] = nombre
                    info["Apellidos"] = apellidos
                else:
                    # Si no, limpia los nombres extraídos por OCR
                    info["Nombre"] = limpiar_texto(nombre_ocr)
                    info["Apellidos"] = limpiar_texto(apellidos_ocr)
                if grupo == "extraviado" and grupo_ocr:
                    info["Grupo_detectado"] = grupo_ocr
                return info, None
            except json.JSONDecodeError as e:
                return None, f"Error parseando JSON: {e}\nRespuesta recibida: {info_str}"
        else:
            return None, f"Error en API OpenAI: {response.status_code} - {response.text}"
    except Exception as e:
        return None, f"Error extrayendo información: {e}"

def extraer_solo_practica(base64_image, headers):
    try:
        prompt_practica = (
            "Look at this exam image and determine if this is a \"Práctica de Listas\" (practice 3) "
            "or \"Práctica de Grafos\" (practice 5) based on the content. "
            "Return JSON with fields: \"Apellidos\": \"\", \"Nombre\": \"\", \"Grupo\": \"extraviado\", \"practica_detectada\": 3 or 5"
        )
        payload = {
            "model": "gpt-4o",
            "response_format": {"type": "json_object"},
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant that identifies exam types and outputs JSON.",
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_practica},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                    ]
                }
            ],
            "max_tokens": 150
        }
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload,
                timeout=10
            )
        except requests.exceptions.Timeout:
            return None, "Timeout: La petición a OpenAI tardó más de 10 segundos (solo práctica)."
        except requests.exceptions.RequestException as e:
            return None, f"Error de conexión con OpenAI (solo práctica): {e}"

        if response.status_code == 200:
            info_str = response.json()['choices'][0]['message']['content']
            try:
                info = json.loads(info_str)
                return info, None
            except json.JSONDecodeError:
                return None, f"Error parseando JSON (solo práctica): {info_str}"
        else:
            return None, f"Error en API OpenAI (solo práctica): {response.status_code} - {response.text}"
    except Exception as e:
        return None, f"Error en retry de práctica: {e}"

def extraer_numero_lote(nombre):
    match = re.search(r"lote_(\d+)", nombre)
    return int(match.group(1)) if match else 0

def mover_archivo_organizado(archivo_original, info, output_path):
    try:
        practica = info.get('practica_detectada', 'desconocida')
        grupo = info.get('Grupo', 'extraviado')
        carpeta_grupo = output_path / grupo
        carpeta_practica = carpeta_grupo / f"Practica_{practica}"
        carpeta_practica.mkdir(parents=True, exist_ok=True)
        apellidos = info.get('Apellidos', '').strip()
        nombre = info.get('Nombre', '').strip()
        if grupo == "extraviado" and not apellidos and not nombre:
            nombre_base = Path(info.get('archivo_original', archivo_original.name)).stem
        else:
            apellidos = apellidos if apellidos else 'SinApellidos'
            nombre = nombre if nombre else 'SinNombre'
            nombre_base = f"{apellidos.replace(' ', '_')}_{nombre.replace(' ', '_')}"
            if grupo == "extraviado" and info.get('Grupo_detectado'):
                nombre_base += f"_{info['Grupo_detectado']}"
        extension = archivo_original.suffix
        nuevo_archivo = carpeta_practica / f"{nombre_base}{extension}"
        contador = 2
        while nuevo_archivo.exists():
            nuevo_archivo = carpeta_practica / f"{nombre_base}_{contador}{extension}"
            contador += 1
        shutil.copy2(archivo_original, nuevo_archivo)
        print(f"  → Guardado en: {carpeta_grupo.name}/{carpeta_practica.name}/{nuevo_archivo.name}")
    except Exception as e:
        print(f"Error organizando archivo {archivo_original.name}: {e}")

def crear_dataframe_examenes(examenes_info, df):
    df_base = df[['Nombre', 'Apellido(s)', 'Grupos']].copy()
    df_base['Examen_3'] = 0
    df_base['Comentario_Examen_3'] = 'PNP'
    df_base['Examen_5'] = 0
    df_base['Comentario_Examen_5'] = 'PNP'
    for examen in examenes_info:
        practica = examen.get('practica_detectada')
        if practica in [3, 5, '3', '5']:
            practica = str(practica)
            apellidos = examen.get('Apellidos', '').upper().strip()
            nombre = examen.get('Nombre', '').upper().strip()
            if apellidos and nombre:
                mask = df_base['Apellido(s)'].str.upper().str.strip() == apellidos
                mask &= df_base['Nombre'].str.upper().str.strip() == nombre
                if mask.any():
                    df_base.loc[mask, f'Examen_{practica}'] = 1
                    df_base.loc[mask, f'Comentario_Examen_{practica}'] = ''
                    print(f"  → Marcado como presentado: {nombre} {apellidos} - Práctica {practica}")
    return df_base

def procesar_examenes_completo(carpeta_examenes="../data/saved/", output_dir="../data/examenes_procesados/"):
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    extraviados_path = output_path / "extraviados"
    extraviados_path.mkdir(parents=True, exist_ok=True)
    examenes_info = []
    errores_openai = []
    # Ordenar por número de lote, no alfabéticamente
    pdf_files = sorted(Path(carpeta_examenes).glob("*.pdf"), key=lambda x: extraer_numero_lote(x.name))
    print(f"Procesando {len(pdf_files)} exámenes...")
    api_key = os.environ["OPENAI_API_KEY"]
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    # Cargar DataFrame de alumnos
    students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
    dfs = [ pd.read_csv(filename) for filename in students_info ]
    df = pd.concat(dfs, ignore_index=True)
    df["Nombre"] = df["Nombre"].str.strip().str.upper()
    df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
    for pdf_file in pdf_files:
        print(f"\nProcesando: {pdf_file.name}")
        try:
            reader = PdfReader(pdf_file)
            num_pages = len(reader.pages)
            info_extraida = None
            error_detalle = None
            for page_num in range(min(2, num_pages)):
                images = convert_from_path(
                    pdf_file,
                    first_page=page_num + 1,
                    last_page=page_num + 1,
                    dpi=150,
                    fmt='jpeg'
                )
                if not images:
                    continue
                buffered = io.BytesIO()
                images[0].save(buffered, format="JPEG")
                base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
                info, error = extraer_info_con_openai(base64_image, headers, df)
                if info and info.get('Apellidos') and info.get('Nombre'):
                    info['archivo_original'] = pdf_file.name
                    info['pagina'] = page_num + 1
                    info_extraida = info
                    break
                elif info and not info.get('Apellidos'):
                    info_retry, error_retry = extraer_solo_practica(base64_image, headers)
                    if info_retry:
                        info_extraida = info_retry
                        info_extraida['archivo_original'] = pdf_file.name
                        info_extraida['pagina'] = page_num + 1
                        break
                    elif error_retry:
                        error_detalle = error_retry
                elif error:
                    error_detalle = error
            if info_extraida:
                mover_archivo_organizado(pdf_file, info_extraida, output_path)
                examenes_info.append(info_extraida)
                practica = info_extraida.get('practica_detectada', 'desconocida')
                print(f"✓ Extraído: {info_extraida.get('Nombre', 'Sin nombre')} {info_extraida.get('Apellidos', 'Sin apellidos')} - Grupo: {info_extraida.get('Grupo', 'extraviado')} - Práctica: {practica}")
            else:
                shutil.copy2(pdf_file, extraviados_path / pdf_file.name)
                print(f"✗ No se pudo extraer información - Copiado a extraviados: {pdf_file.name}")
                errores_openai.append({
                    "archivo": pdf_file.name,
                    "error": error_detalle or "No se pudo extraer información ni con retry",
                })
        except Exception as e:
            print(f"✗ Error procesando {pdf_file.name}: {e}")
            try:
                shutil.copy2(pdf_file, extraviados_path / pdf_file.name)
                print(f"  → Copiado a extraviados por error")
            except:
                pass
            errores_openai.append({
                "archivo": pdf_file.name,
                "error": str(e),
            })
            continue
    df_examenes = crear_dataframe_examenes(examenes_info, df)
    df_examenes.to_csv(output_path / "seguimiento_examenes.csv", index=False)
    print(f"\n📊 Procesamiento completado:")
    print(f"- Exámenes procesados: {len(examenes_info)}")
    print(f"- Archivos en extraviados: {len(list(extraviados_path.glob('*.pdf')))}")
    print(f"- Archivo de seguimiento guardado en: {output_path / 'seguimiento_examenes.csv'}")
    # Crear DataFrame de errores y mostrarlo
    if errores_openai:
        df_errores = pd.DataFrame(errores_openai)
        df_errores.to_csv(output_path / "errores_openai.csv", index=False)
        print(f"\n❗ Casos fallidos por OpenAI guardados en: {output_path / 'errores_openai.csv'}")
        print(df_errores)
    else:
        print("\n✅ No hubo errores de extracción con OpenAI.")
    return df_examenes

In [26]:
# Ejecutar el procesamiento
#df_examenes_procesados = procesar_examenes_completo()
print("\n📊 Resumen de exámenes procesados por grupo:")
#print(df_examenes_procesados.groupby('Grupos')[['Examen_3', 'Examen_5']].sum())


📊 Resumen de exámenes procesados por grupo:


In [ ]:
import os
import shutil
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Text, Dropdown, Checkbox
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import re
import matplotlib.pyplot as plt

class JupyterExamReviewer:
    def __init__(self, examenes_dir="../data/examenes_procesados/"):
        self.examenes_dir = Path(examenes_dir)
        self.current_index = 0
        self.exam_files = []
        self.changes_pending = {}  # Diccionario para almacenar cambios pendientes
        self.out = Output()
        self.status = Label(value="Cargando exámenes...")
        
        # Cargar todos los archivos PDF
        self._load_exam_files()
        
        if not self.exam_files:
            self.status.value = "No se encontraron exámenes para revisar"
            return
            
        # Configurar widgets
        self._setup_widgets()
        self._show_current_exam()

    def _load_exam_files(self):
        """Carga todos los archivos PDF de todas las carpetas de grupos y prácticas"""
        print("🔍 Escaneando carpetas de exámenes...")
        
        for grupo_dir in self.examenes_dir.iterdir():
            if grupo_dir.is_dir() and grupo_dir.name not in ["extraviados", "problemático"]:
                for practica_dir in grupo_dir.iterdir():
                    if practica_dir.is_dir() and practica_dir.name.startswith("Practica_"):
                        for pdf_file in practica_dir.glob("*.pdf"):
                            self.exam_files.append({
                                'file_path': pdf_file,
                                'grupo_actual': grupo_dir.name,
                                'practica': practica_dir.name,
                                'nombre_archivo': pdf_file.stem
                            })
        
        # Ordenar por grupo y luego por nombre
        self.exam_files.sort(key=lambda x: (x['grupo_actual'], x['nombre_archivo']))
        print(f"📁 Total de exámenes encontrados: {len(self.exam_files)}")

    def _setup_widgets(self):
        """Configura todos los widgets de la interfaz"""
        # Botones de navegación
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='120px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='120px'))
        
        # Campos de edición
        self.txt_nombre = Text(description='Nombre:', layout=Layout(width='300px'))
        self.txt_apellidos = Text(description='Apellidos:', layout=Layout(width='300px'))
        
        # Dropdown para cambiar grupo
        grupos_disponibles = [d.name for d in self.examenes_dir.iterdir() 
                             if d.is_dir() and d.name not in ["extraviados", "problemático"]]
        grupos_disponibles.extend(["extraviados", "problemático"])  # Añadir opciones especiales
        
        self.dropdown_grupo = Dropdown(
            options=grupos_disponibles,
            description='Grupo:',
            layout=Layout(width='200px')
        )
        
        # Checkboxes para confirmar carpetas correctas
        self.check_nombre_ok = Checkbox(description='Nombre correcto', value=True)
        self.check_grupo_ok = Checkbox(description='Grupo correcto', value=True)
        
        # Botón para aplicar cambios
        self.btn_apply = Button(
            description='✓ Aplicar Cambios',
            button_style='success',
            layout=Layout(width='150px')
        )
        
        # Contador de progreso
        self.progress_label = Label(value="")
        
        # Conectar eventos
        self.btn_prev.on_click(lambda x: self._navigate(-1))
        self.btn_next.on_click(lambda x: self._navigate(1))
        self.btn_apply.on_click(lambda x: self._apply_changes())
        
        # Layout de la interfaz
        controls_row1 = HBox([
            self.btn_prev, 
            self.progress_label,
            self.btn_next
        ])
        
        edit_row = HBox([
            self.txt_nombre,
            self.txt_apellidos,
            self.dropdown_grupo
        ])
        
        check_row = HBox([
            self.check_nombre_ok,
            self.check_grupo_ok,
            self.btn_apply
        ])
        
        self.interface = VBox([
            controls_row1,
            edit_row,
            check_row,
            self.status,
            self.out
        ])
        
        display(self.interface)

    def _show_current_exam(self):
        """Muestra el examen actual"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        
        # Actualizar contador de progreso
        self.progress_label.value = f"Examen {self.current_index + 1} de {len(self.exam_files)}"
        
        # Actualizar campos con información actual
        nombre_completo = current_exam['nombre_archivo']
        partes = nombre_completo.replace('_', ' ').split()
        
        if len(partes) >= 2:
            self.txt_apellidos.value = ' '.join(partes[:-1])
            self.txt_nombre.value = partes[-1]
        else:
            self.txt_apellidos.value = nombre_completo
            self.txt_nombre.value = ""
            
        self.dropdown_grupo.value = current_exam['grupo_actual']
        
        # Resetear checkboxes
        self.check_nombre_ok.value = True
        self.check_grupo_ok.value = True
        
        # Mostrar información del archivo
        self.status.value = f"📁 {current_exam['grupo_actual']} / {current_exam['practica']} / {current_exam['nombre_archivo']}.pdf"
        
        # Mostrar imagen del examen
        self._show_exam_image(current_exam['file_path'])

    def _show_exam_image(self, pdf_path):
        """Muestra las páginas del PDF (hasta 2 páginas)"""
        with self.out:
            clear_output(wait=True)
            try:
                # Contar páginas primero
                reader = PdfReader(pdf_path)
                num_pages = len(reader.pages)
                
                # Si el PDF no tiene exactamente 2 páginas, ofrecer opción "problemático"
                if num_pages != 2:
                    print(f"⚠️ El PDF tiene {num_pages} página(s) en lugar de 2")
                    print("💡 Considera moverlo a la carpeta 'problemático'")
                    # Actualizar dropdown para mostrar "problemático" como opción sugerida
                    if "problemático" not in [opt for opt in self.dropdown_grupo.options]:
                        self.dropdown_grupo.value = "problemático"
                
                # Convertir hasta 2 páginas
                pages_to_show = min(2, num_pages)
                images = convert_from_path(
                    pdf_path,
                    first_page=1,
                    last_page=pages_to_show,
                    dpi=150,
                    fmt='jpeg'
                )
                
                if images:
                    # Configurar subplots según el número de imágenes
                    if len(images) == 1:
                        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
                        axes = [ax]
                    else:
                        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
                    
                    for i, image in enumerate(images):
                        # Recortar la parte superior (donde están los datos del estudiante)
                        if i == 0:  # Primera página - cabecera
                            cropped = image.crop((0, 0, image.width, min(600, image.height // 3)))
                            title = f"Página {i+1} - Cabecera"
                        else:  # Segunda página - completa pero más pequeña
                            cropped = image.crop((0, 0, image.width, image.height))
                            title = f"Página {i+1} - Completa"
                        
                        axes[i].imshow(cropped)
                        axes[i].axis('off')
                        axes[i].set_title(title)
                    
                    plt.suptitle(f"Examen: {pdf_path.name} ({num_pages} página(s))")
                    plt.tight_layout()
                    plt.show()
                else:
                    print("❌ No se pudo cargar la imagen del PDF")
                    
            except Exception as e:
                print(f"❌ Error cargando imagen: {e}")

    def _navigate(self, direction):
        """Navega al examen anterior o siguiente"""
        # Aplicar cambios pendientes si los hay
        if self._has_pending_changes():
            self._apply_changes()
        
        # Cambiar índice
        new_index = self.current_index + direction
        
        if 0 <= new_index < len(self.exam_files):
            self.current_index = new_index
            self._show_current_exam()
        elif new_index >= len(self.exam_files):
            self.status.value = "🎉 ¡Revisión completada! Has llegado al final."
        elif new_index < 0:
            self.status.value = "📍 Ya estás en el primer examen."

    def _has_pending_changes(self):
        """Verifica si hay cambios pendientes"""
        if not self.exam_files:
            return False
            
        current_exam = self.exam_files[self.current_index]
        
        # Verificar cambios en nombre/apellidos
        nombre_actual = self.txt_nombre.value.strip().upper()
        apellidos_actual = self.txt_apellidos.value.strip().upper()
        nuevo_nombre = f"{apellidos_actual}_{nombre_actual}".replace(' ', '_')
        
        nombre_archivo_actual = current_exam['nombre_archivo']
        
        # Verificar cambio de grupo
        grupo_cambio = self.dropdown_grupo.value != current_exam['grupo_actual']
        nombre_cambio = nuevo_nombre != nombre_archivo_actual
        
        return grupo_cambio or nombre_cambio or not self.check_nombre_ok.value or not self.check_grupo_ok.value

    def _apply_changes(self):
        """Aplica los cambios al archivo actual"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        old_path = current_exam['file_path']
        
        try:
            # Preparar nuevo nombre de archivo
            nombre_nuevo = self.txt_nombre.value.strip().replace(' ', '_')
            apellidos_nuevo = self.txt_apellidos.value.strip().replace(' ', '_')
            nuevo_nombre_archivo = f"{apellidos_nuevo}_{nombre_nuevo}"
            
            # Preparar nueva ubicación
            nuevo_grupo = self.dropdown_grupo.value
            practica_actual = current_exam['practica']
            
            nueva_carpeta = self.examenes_dir / nuevo_grupo
            if nuevo_grupo not in ["extraviados", "problemático"]:
                nueva_carpeta = nueva_carpeta / practica_actual
            
            # Crear carpeta si no existe
            nueva_carpeta.mkdir(parents=True, exist_ok=True)
            
            # Nuevo path completo
            nuevo_path = nueva_carpeta / f"{nuevo_nombre_archivo}.pdf"
            
            # Evitar conflictos de nombres
            contador = 2
            while nuevo_path.exists() and nuevo_path != old_path:
                nuevo_path = nueva_carpeta / f"{nuevo_nombre_archivo}_{contador}.pdf"
                contador += 1
            
            # Mover archivo si es necesario
            if nuevo_path != old_path:
                shutil.move(str(old_path), str(nuevo_path))
                
                # Actualizar información en la lista
                current_exam['file_path'] = nuevo_path
                current_exam['grupo_actual'] = nuevo_grupo
                current_exam['nombre_archivo'] = nuevo_path.stem
                
                self.status.value = f"✅ Movido a: {nuevo_grupo}/{nuevo_path.name}"
            else:
                self.status.value = "ℹ️ Sin cambios necesarios"
                
        except Exception as e:
            self.status.value = f"❌ Error aplicando cambios: {e}"

def iniciar_revision_examenes(examenes_dir="../data/examenes_procesados/"):
    """Inicia la interfaz de revisión de exámenes"""
    print("🚀 Iniciando revisor de exámenes...")
    reviewer = JupyterExamReviewer(examenes_dir)
    return reviewer


In [ ]:
reviewer = iniciar_revision_examenes()

In [ ]:
import os
import shutil
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Text, Dropdown, Checkbox
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import re

import matplotlib.pyplot as plt

class JupyterExamReviewer:
    def __init__(self, examenes_dir="../data/examenes_procesados/"):
        self.examenes_dir = Path(examenes_dir)
        self.current_index = 0
        self.exam_files = []
        self.changes_pending = {}  # Diccionario para almacenar cambios pendientes
        self.out = Output()
        self.status = Label(value="Cargando exámenes...")
        
        # Cargar todos los archivos PDF
        self._load_exam_files()
        
        if not self.exam_files:
            self.status.value = "No se encontraron exámenes para revisar"
            return
            
        # Configurar widgets
        self._setup_widgets()
        self._show_current_exam()

    def _load_exam_files(self):
        """Carga todos los archivos PDF de todas las carpetas de grupos y prácticas"""
        print("🔍 Escaneando carpetas de exámenes...")
        
        for grupo_dir in self.examenes_dir.iterdir():
            if grupo_dir.is_dir() and grupo_dir.name not in ["extraviados", "problemático"]:
                for practica_dir in grupo_dir.iterdir():
                    if practica_dir.is_dir() and practica_dir.name.startswith("Practica_"):
                        for pdf_file in practica_dir.glob("*.pdf"):
                            self.exam_files.append({
                                'file_path': pdf_file,
                                'grupo_actual': grupo_dir.name,
                                'practica': practica_dir.name,
                                'nombre_archivo': pdf_file.stem
                            })
        
        # También cargar archivos de carpetas especiales
        for special_dir in ["extraviados", "problemático"]:
            special_path = self.examenes_dir / special_dir
            if special_path.exists():
                for pdf_file in special_path.glob("*.pdf"):
                    self.exam_files.append({
                        'file_path': pdf_file,
                        'grupo_actual': special_dir,
                        'practica': "",  # Sin práctica para carpetas especiales
                        'nombre_archivo': pdf_file.stem
                    })
        
        # Ordenar por grupo y luego por nombre
        self.exam_files.sort(key=lambda x: (x['grupo_actual'], x['nombre_archivo']))
        print(f"📁 Total de exámenes encontrados: {len(self.exam_files)}")

    def _setup_widgets(self):
        """Configura todos los widgets de la interfaz"""
        # Botones de navegación
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='120px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='120px'))
        
        # Campos de edición
        self.txt_nombre = Text(description='Nombre:', layout=Layout(width='300px'))
        self.txt_apellidos = Text(description='Apellidos:', layout=Layout(width='300px'))
        
        # Dropdown para cambiar grupo
        grupos_disponibles = [d.name for d in self.examenes_dir.iterdir() 
                             if d.is_dir() and d.name not in ["extraviados", "problemático"]]
        grupos_disponibles.extend(["extraviados", "problemático"])  # Añadir opciones especiales
        
        self.dropdown_grupo = Dropdown(
            options=grupos_disponibles,
            description='Grupo:',
            layout=Layout(width='200px')
        )
        
        # Dropdown para cambiar práctica
        self.dropdown_practica = Dropdown(
            options=['2', '3', '4', '5'],
            description='Práctica:',
            layout=Layout(width='150px')
        )
        
        # Checkboxes para confirmar carpetas correctas
        self.check_nombre_ok = Checkbox(description='Nombre correcto', value=True)
        self.check_grupo_ok = Checkbox(description='Grupo correcto', value=True)
        
        # Botón para aplicar cambios
        self.btn_apply = Button(
            description='✓ Aplicar Cambios',
            button_style='success',
            layout=Layout(width='150px')
        )
        
        # Contador de progreso
        self.progress_label = Label(value="")
        
        # Conectar eventos
        self.btn_prev.on_click(lambda x: self._navigate(-1))
        self.btn_next.on_click(lambda x: self._navigate(1))
        self.btn_apply.on_click(lambda x: self._apply_changes())
        self.dropdown_grupo.observe(self._on_grupo_change, names='value')
        
        # Layout de la interfaz
        controls_row1 = HBox([
            self.btn_prev, 
            self.progress_label,
            self.btn_next
        ])
        
        edit_row = HBox([
            self.txt_nombre,
            self.txt_apellidos,
            self.dropdown_grupo,
            self.dropdown_practica
        ])
        
        check_row = HBox([
            self.check_nombre_ok,
            self.check_grupo_ok,
            self.btn_apply
        ])
        
        self.interface = VBox([
            controls_row1,
            edit_row,
            check_row,
            self.status,
            self.out
        ])
        
        display(self.interface)

    def _on_grupo_change(self, change):
        """Maneja el cambio de grupo para habilitar/deshabilitar dropdown de práctica"""
        if change['new'] in ["extraviados", "problemático"]:
            self.dropdown_practica.disabled = True
        else:
            self.dropdown_practica.disabled = False

    def _show_current_exam(self):
        """Muestra el examen actual"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        
        # Actualizar contador de progreso
        self.progress_label.value = f"Examen {self.current_index + 1} de {len(self.exam_files)}"
        
        # Actualizar campos con información actual
        nombre_completo = current_exam['nombre_archivo']
        partes = nombre_completo.replace('_', ' ').split()
        
        if len(partes) >= 2:
            self.txt_apellidos.value = ' '.join(partes[:-1])
            self.txt_nombre.value = partes[-1]
        else:
            self.txt_apellidos.value = nombre_completo
            self.txt_nombre.value = ""
            
        self.dropdown_grupo.value = current_exam['grupo_actual']
        
        # Extraer número de práctica actual
        if current_exam['practica']:
            practica_num = current_exam['practica'].replace('Practica_', '')
            if practica_num in ['2', '3', '4', '5']:
                self.dropdown_practica.value = practica_num
            else:
                self.dropdown_practica.value = '3'  # valor por defecto
        else:
            self.dropdown_practica.value = '3'  # valor por defecto para carpetas especiales
        
        # Habilitar/deshabilitar dropdown de práctica
        if current_exam['grupo_actual'] in ["extraviados", "problemático"]:
            self.dropdown_practica.disabled = True
        else:
            self.dropdown_practica.disabled = False
        
        # Resetear checkboxes
        self.check_nombre_ok.value = True
        self.check_grupo_ok.value = True
        
        # Mostrar información del archivo
        if current_exam['practica']:
            self.status.value = f"📁 {current_exam['grupo_actual']} / {current_exam['practica']} / {current_exam['nombre_archivo']}.pdf"
        else:
            self.status.value = f"📁 {current_exam['grupo_actual']} / {current_exam['nombre_archivo']}.pdf"
        
        # Mostrar imagen del examen
        self._show_exam_image(current_exam['file_path'])

    def _show_exam_image(self, pdf_path):
        """Muestra las páginas del PDF (hasta 2 páginas)"""
        with self.out:
            clear_output(wait=True)
            try:
                # Contar páginas primero
                reader = PdfReader(pdf_path)
                num_pages = len(reader.pages)
                
                # Si el PDF no tiene exactamente 2 páginas, ofrecer opción "problemático"
                if num_pages != 2:
                    print(f"⚠️ El PDF tiene {num_pages} página(s) en lugar de 2")
                    print("💡 Considera moverlo a la carpeta 'problemático'")
                
                # Convertir hasta 2 páginas
                pages_to_show = min(2, num_pages)
                images = convert_from_path(
                    pdf_path,
                    first_page=1,
                    last_page=pages_to_show,
                    dpi=150,
                    fmt='jpeg'
                )
                
                if images:
                    # Configurar subplots según el número de imágenes
                    if len(images) == 1:
                        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
                        axes = [ax]
                    else:
                        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
                    
                    for i, image in enumerate(images):
                        # Recortar la parte superior (donde están los datos del estudiante)
                        if i == 0:  # Primera página - cabecera
                            cropped = image.crop((0, 0, image.width, min(600, image.height // 3)))
                            title = f"Página {i+1} - Cabecera"
                        else:  # Segunda página - completa pero más pequeña
                            cropped = image.crop((0, 0, image.width, image.height))
                            title = f"Página {i+1} - Completa"
                        
                        axes[i].imshow(cropped)
                        axes[i].axis('off')
                        axes[i].set_title(title)
                    
                    plt.suptitle(f"Examen: {pdf_path.name} ({num_pages} página(s))")
                    plt.tight_layout()
                    plt.show()
                else:
                    print("❌ No se pudo cargar la imagen del PDF")
                    
            except Exception as e:
                print(f"❌ Error cargando imagen: {e}")

    def _navigate(self, direction):
        """Navega al examen anterior o siguiente"""
        # Aplicar cambios pendientes si los hay
        if self._has_pending_changes():
            self._apply_changes()
        
        # Cambiar índice
        new_index = self.current_index + direction
        
        if 0 <= new_index < len(self.exam_files):
            self.current_index = new_index
            self._show_current_exam()
        elif new_index >= len(self.exam_files):
            self.status.value = "🎉 ¡Revisión completada! Has llegado al final."
        elif new_index < 0:
            self.status.value = "📍 Ya estás en el primer examen."

    def _has_pending_changes(self):
        """Verifica si hay cambios pendientes"""
        if not self.exam_files:
            return False
            
        current_exam = self.exam_files[self.current_index]
        
        # Verificar cambios en nombre/apellidos
        nombre_actual = self.txt_nombre.value.strip().upper()
        apellidos_actual = self.txt_apellidos.value.strip().upper()
        nuevo_nombre = f"{apellidos_actual}_{nombre_actual}".replace(' ', '_')
        
        nombre_archivo_actual = current_exam['nombre_archivo']
        
        # Verificar cambio de grupo
        grupo_cambio = self.dropdown_grupo.value != current_exam['grupo_actual']
        
        # Verificar cambio de práctica
        practica_actual = current_exam['practica'].replace('Practica_', '') if current_exam['practica'] else ''
        practica_cambio = self.dropdown_practica.value != practica_actual and not self.dropdown_practica.disabled
        
        nombre_cambio = nuevo_nombre != nombre_archivo_actual
        
        return grupo_cambio or practica_cambio or nombre_cambio or not self.check_nombre_ok.value or not self.check_grupo_ok.value

    def _apply_changes(self):
        """Aplica los cambios al archivo actual"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        old_path = current_exam['file_path']
        
        try:
            # Preparar nuevo nombre de archivo
            nombre_nuevo = self.txt_nombre.value.strip().replace(' ', '_')
            apellidos_nuevo = self.txt_apellidos.value.strip().replace(' ', '_')
            nuevo_nombre_archivo = f"{apellidos_nuevo}_{nombre_nuevo}"
            
            # Preparar nueva ubicación
            nuevo_grupo = self.dropdown_grupo.value
            nueva_practica = self.dropdown_practica.value
            
            nueva_carpeta = self.examenes_dir / nuevo_grupo
            
            # Solo añadir carpeta de práctica si no es una carpeta especial
            if nuevo_grupo not in ["extraviados", "problemático"]:
                nueva_carpeta = nueva_carpeta / f"Practica_{nueva_practica}"
            
            # Crear carpeta si no existe
            nueva_carpeta.mkdir(parents=True, exist_ok=True)
            
            # Nuevo path completo
            nuevo_path = nueva_carpeta / f"{nuevo_nombre_archivo}.pdf"
            
            # Evitar conflictos de nombres
            contador = 2
            while nuevo_path.exists() and nuevo_path != old_path:
                nuevo_path = nueva_carpeta / f"{nuevo_nombre_archivo}_{contador}.pdf"
                contador += 1
            
            # Mover archivo si es necesario
            if nuevo_path != old_path:
                shutil.move(str(old_path), str(nuevo_path))
                
                # Actualizar información en la lista
                current_exam['file_path'] = nuevo_path
                current_exam['grupo_actual'] = nuevo_grupo
                current_exam['practica'] = f"Practica_{nueva_practica}" if nuevo_grupo not in ["extraviados", "problemático"] else ""
                current_exam['nombre_archivo'] = nuevo_path.stem
                
                if nuevo_grupo in ["extraviados", "problemático"]:
                    self.status.value = f"✅ Movido a: {nuevo_grupo}/{nuevo_path.name}"
                else:
                    self.status.value = f"✅ Movido a: {nuevo_grupo}/Practica_{nueva_practica}/{nuevo_path.name}"
            else:
                self.status.value = "ℹ️ Sin cambios necesarios"
                
        except Exception as e:
            self.status.value = f"❌ Error aplicando cambios: {e}"

def iniciar_revision_examenes(examenes_dir="../data/examenes_procesados/"):
    """Inicia la interfaz de revisión de exámenes"""
    print("🚀 Iniciando revisor de exámenes...")
    reviewer = JupyterExamReviewer(examenes_dir)
    return reviewer

In [ ]:
#reviewer = iniciar_revision_examenes()

In [52]:
import os
import shutil
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Combobox, Dropdown, HTML, ToggleButtons
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd
from difflib import get_close_matches
import re

# --- Cargar el DataFrame de alumnos (ajusta la ruta si es necesario) ---
students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
dfs = [pd.read_csv(filename) for filename in students_info]
df = pd.concat(dfs, ignore_index=True)
df["Nombre"] = df["Nombre"].str.strip().str.upper()
df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
df["Grupos"] = df["Grupos"].astype(str).str.strip()

# Lista de nombres completos para autocompletar
nombres_completos = [
    f"{row['Apellido(s)']} {row['Nombre']}" for _, row in df.iterrows()
]
nombre_a_grupo = {
    f"{row['Apellido(s)']} {row['Nombre']}": row['Grupos'] for _, row in df.iterrows()
}

class JupyterExamReviewer:
    def __init__(self, examenes_dir="../data/examenes_procesados/"):
        self.examenes_dir = Path(examenes_dir)
        self.current_index = 0
        self.exam_files = []
        self.out = Output()
        self.status = Label(value="Cargando exámenes...")

        # Cargar todos los archivos PDF
        self._load_exam_files()

        if not self.exam_files:
            self.status.value = "No se encontraron exámenes para revisar"
            return

        # Configurar widgets
        self._setup_widgets()
        self._show_current_exam()

    def _load_exam_files(self):
        print("🔍 Escaneando carpetas de exámenes...")
        for grupo_dir in self.examenes_dir.iterdir():
            if grupo_dir.is_dir():
                for practica_dir in grupo_dir.iterdir():
                    if practica_dir.is_dir():
                        for pdf_file in practica_dir.glob("*.pdf"):
                            self.exam_files.append({
                                'file_path': pdf_file,
                                'carpeta_actual': grupo_dir.name,
                                'practica': practica_dir.name if practica_dir.name.startswith("Practica_") else "",
                                'nombre_archivo': pdf_file.stem
                            })
                # También incluir PDFs sueltos en la carpeta (por si acaso)
                for pdf_file in grupo_dir.glob("*.pdf"):
                    self.exam_files.append({
                        'file_path': pdf_file,
                        'carpeta_actual': grupo_dir.name,
                        'practica': "",
                        'nombre_archivo': pdf_file.stem
                    })
        self.exam_files.sort(key=lambda x: (x['carpeta_actual'], x['nombre_archivo']))
        print(f"📁 Total de exámenes encontrados: {len(self.exam_files)}")

    def _setup_widgets(self):
        # Status (ruta de archivo) - arriba
        self.status.layout = Layout(width='100%')
        
        # Nombre del alumno (principal)
        self.combo_nombre = Combobox(
            placeholder='Escribe o selecciona...',
            options=nombres_completos,
            description='Alumno:',
            layout=Layout(width='450px')
        )
        self.combo_nombre.observe(self._on_nombre_change, names='value')

        # Botón de aplicar cambios
        self.btn_apply = Button(
            description='✓ Aplicar',
            button_style='success',
            layout=Layout(width='80px')
        )
        
        # Botón para eliminar archivo (NUEVO)
        self.btn_delete = Button(
            description='🗑️ Eliminar',
            button_style='danger',
            layout=Layout(width='90px')
        )
        
        # Contador de progreso
        self.progress_label = Label(value="", layout=Layout(width='140px'))

        # Widgets de grupo, práctica y carpeta
        grupos_unicos = sorted(df["Grupos"].unique())
        self.dropdown_grupo = Dropdown(
            options=grupos_unicos,
            description='Grupo:',
            layout=Layout(width='280px')  # Más ancho para que se vea bien
        )

        self.dropdown_practica = Dropdown(
            options=['2', '3', '4', '5'],
            description='Práctica:',
            layout=Layout(width='180px')
        )

        # Selector de carpeta destino
        carpetas_disponibles = [d.name for d in self.examenes_dir.iterdir() if d.is_dir()]
        self.dropdown_carpeta = Dropdown(
            options=carpetas_disponibles,
            description='Carpeta:',
            layout=Layout(width='170px')
        )
        self.dropdown_carpeta.observe(self._on_carpeta_change, names='value')

        # HTML para mostrar sugerencias de nombres similares (horizontal)
        self.similares_html = HTML(value="", layout=Layout(width='100%', max_height='60px'))

        # Botones de navegación 
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='120px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='120px'))
        
        # Selector de página ajustado (botones a la derecha del texto)
        self.page_label = Label(value="Ver pág:", layout=Layout(width='50px'))
        self.page_selector = ToggleButtons(
            options=[('1', 1), ('2', 2)],
            value=1,
            description='',  # Quitamos la descripción del control y la ponemos separada
            style={'button_width': '30px'},
            layout=Layout(width='80px')
        )
        
        self.page_selector.observe(self._on_page_change, names='value')

        # Conectar eventos
        self.btn_prev.on_click(lambda x: self._navigate(-1))
        self.btn_next.on_click(lambda x: self._navigate(1))
        self.btn_apply.on_click(lambda x: self._apply_changes())
        self.btn_delete.on_click(lambda x: self._delete_current_file())  # NUEVO

        # REORGANIZACIÓN DE LA INTERFAZ:
        
        # Fila 1: Path/status (menos importante, arriba)
        status_row = HBox([self.status])
        
        # Fila 2: Nombre + aplicar + eliminar + progreso (MODIFICADO)
        nombre_row = HBox([
            self.combo_nombre,
            self.btn_apply,
            self.btn_delete,  # NUEVO
            self.progress_label
        ])
        
        # Fila 3: Grupo, práctica, carpeta (más visibles)
        opciones_row = HBox([
            self.dropdown_grupo,
            self.dropdown_practica, 
            self.dropdown_carpeta
        ])
        
        # Fila 4: Sugerencias similares (a lo ancho)
        sugerencias_row = HBox([self.similares_html])
        
        # Fila 5: Navegación (abajo) + selector de páginas en línea
        page_selector_group = HBox([
            self.page_label, 
            self.page_selector
            ], 
            layout=Layout(width='140px')
        )
        nav_row = HBox([
            self.btn_prev,
            self.btn_next,
            page_selector_group
        ])  

        # Layout general
        self.interface = VBox([
            status_row,
            nombre_row,
            opciones_row,
            sugerencias_row,
            nav_row,
            self.out
        ])

        display(self.interface)

    def _delete_current_file(self):
        """Elimina el archivo actual (NUEVO MÉTODO)"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        file_path = current_exam['file_path']
        
        try:
            # Crear carpeta de eliminados si no existe
            deleted_folder = self.examenes_dir / "eliminados"
            deleted_folder.mkdir(exist_ok=True)
            
            # Mover a la carpeta de eliminados en lugar de eliminar permanentemente
            deleted_path = deleted_folder / file_path.name
            
            # Si ya existe en eliminados, añadir sufijo numérico
            contador = 2
            while deleted_path.exists():
                deleted_path = deleted_folder / f"{file_path.stem}_{contador}{file_path.suffix}"
                contador += 1
            
            # Mover archivo
            shutil.move(str(file_path), str(deleted_path))
            
            # Remover de la lista
            self.exam_files.pop(self.current_index)
            
            # Ajustar índice si es necesario
            if self.current_index >= len(self.exam_files):
                self.current_index = max(0, len(self.exam_files) - 1)
            
            self.status.value = f"🗑️ Eliminado: {file_path.name} → eliminados/{deleted_path.name}"
            
            # Mostrar siguiente examen o mensaje si no hay más
            if self.exam_files:
                self._show_current_exam()
            else:
                self.status.value = "🎉 No hay más exámenes para revisar"
                with self.out:
                    clear_output(wait=True)
                    print("No hay más exámenes para revisar")
                    
        except Exception as e:
            self.status.value = f"❌ Error eliminando archivo: {e}"

    def _on_carpeta_change(self, change):
        # Eliminado: Ya no se deshabilita el dropdown de práctica para ninguna carpeta
        pass

    def _on_nombre_change(self, change):
        valor = change['new']
        if valor in nombre_a_grupo:
            self.dropdown_grupo.value = nombre_a_grupo[valor]
            self.combo_nombre.value = valor
        self._update_similares(valor)

    def _on_page_change(self, change):
        self._show_exam_image(self.exam_files[self.current_index]['file_path'])

    def _update_similares(self, valor):
        if valor:
            matches = get_close_matches(valor, nombres_completos, n=4, cutoff=0)
            # Mostrar horizontalmente (una sola fila)
            html = "<b>Sugerencias:</b> "
            if matches:
                html += "<span style='display:inline-block;white-space:nowrap;'>"
                for i, m in enumerate(matches):
                    html += f"<span style='display:inline-block;margin-right:20px;font-size:90%'>{m} <span style='color:#888;font-size:85%'>({nombre_a_grupo.get(m, '-')})</span></span>"
                html += "</span>"
            else:
                html += "<i>No hay sugerencias</i>"
            self.similares_html.value = html
        else:
            self.similares_html.value = ""

    def _show_current_exam(self):
        if not self.exam_files:
            return

        current_exam = self.exam_files[self.current_index]
        self.progress_label.value = f"Examen {self.current_index + 1} de {len(self.exam_files)}"

        nombre_completo = current_exam['nombre_archivo'].replace('_', ' ')
        mejor_match = None
        for n in nombres_completos:
            if nombre_completo.upper() in n.upper():
                mejor_match = n
                break
        if mejor_match:
            self.combo_nombre.value = mejor_match
            self.dropdown_grupo.value = nombre_a_grupo[mejor_match]
        else:
            self.combo_nombre.value = nombre_completo
            self.dropdown_grupo.value = df["Grupos"].iloc[0]  # valor por defecto

        self._update_similares(self.combo_nombre.value)

        # Práctica
        if current_exam['practica']:
            practica_num = current_exam['practica'].replace('Practica_', '')
            if practica_num in ['2', '3', '4', '5']:
                self.dropdown_practica.value = practica_num
            else:
                self.dropdown_practica.value = '3'
        else:
            self.dropdown_practica.value = '3'

        # Carpeta actual
        self.dropdown_carpeta.value = current_exam['carpeta_actual']

        # Mostrar solo la página 1 por defecto
        self.page_selector.value = 1

        # Ajustar el rango del selector de página según el número de páginas
        reader = PdfReader(current_exam['file_path'])
        num_pages = len(reader.pages)
        if num_pages == 1:
            self.dropdown_carpeta.value = "problemático"
            self.page_selector.disabled = True
        else:
            self.page_selector.disabled = False

        # Eliminado: Ya no se deshabilita el dropdown de práctica

        if current_exam['practica']:
            self.status.value = f"📁 {current_exam['carpeta_actual']} / {current_exam['practica']} / {current_exam['nombre_archivo']}.pdf"
        else:
            self.status.value = f"📁 {current_exam['carpeta_actual']} / {current_exam['nombre_archivo']}.pdf"

        self._show_exam_image(current_exam['file_path'])

    def _show_exam_image(self, pdf_path):
        with self.out:
            clear_output(wait=True)
            try:
                reader = PdfReader(pdf_path)
                num_pages = len(reader.pages)
                page_num = self.page_selector.value
                if num_pages == 1:
                    page_num = 1
                images = convert_from_path(
                    pdf_path,
                    first_page=page_num,
                    last_page=page_num,
                    dpi=150,
                    fmt='jpeg'
                )
                if images:
                    image = images[0]
                    # Recortar más agresivamente por arriba y por abajo (1/4 de la altura)
                    cabecera_altura = int(min(450, image.height // 4))
                    # Recortar un poco desde arriba también (20 píxeles)
                    top_offset = 20
                    cropped = image.crop((0, top_offset, image.width, cabecera_altura))
                    plt.figure(figsize=(10, 6))  # Altura reducida
                    plt.imshow(cropped)
                    plt.axis('off')
                    plt.tight_layout()
                    plt.show()
                else:
                    print("❌ No se pudo cargar la imagen del PDF")
            except Exception as e:
                print(f"❌ Error cargando imagen: {e}")

    def _navigate(self, direction):
        if self._has_pending_changes():
            self._apply_changes()
        new_index = self.current_index + direction
        if 0 <= new_index < len(self.exam_files):
            self.current_index = new_index
            self._show_current_exam()
        elif new_index >= len(self.exam_files):
            self.status.value = "🎉 ¡Revisión completada! Has llegado al final."
        elif new_index < 0:
            self.status.value = "📍 Ya estás en el primer examen."

    def _has_pending_changes(self):
        if not self.exam_files:
            return False
        current_exam = self.exam_files[self.current_index]
        nombre_actual = self.combo_nombre.value.strip().upper().replace(' ', '_')
        nombre_archivo_actual = current_exam['nombre_archivo'].upper()
        carpeta_cambio = self.dropdown_carpeta.value != current_exam['carpeta_actual']
        practica_actual = current_exam['practica'].replace('Practica_', '') if current_exam['practica'] else ''
        # Eliminado: "and not self.dropdown_practica.disabled" ya que nunca se deshabilita
        practica_cambio = self.dropdown_practica.value != practica_actual
        grupo_cambio = self.dropdown_grupo.value != nombre_a_grupo.get(self.combo_nombre.value, self.dropdown_grupo.value)
        nombre_cambio = nombre_actual != nombre_archivo_actual
        return carpeta_cambio or practica_cambio or grupo_cambio or nombre_cambio

    def _apply_changes(self):
        if not self.exam_files:
            return
        current_exam = self.exam_files[self.current_index]
        old_path = current_exam['file_path']
        try:
            nombre_nuevo = self.combo_nombre.value.strip().replace(' ', '_')
            grupo_para_nombre = self.dropdown_grupo.value
            nueva_practica = self.dropdown_practica.value
            carpeta_destino = self.dropdown_carpeta.value

            # Nombre base: APELLIDOS_NOMBRE_P<num practica>_<Grupo>.pdf
            nombre_base = f"{nombre_nuevo}_P{nueva_practica}_{grupo_para_nombre}"
            nueva_carpeta = self.examenes_dir / carpeta_destino
            if carpeta_destino not in ["extraviados", "problemático"]:
                nueva_carpeta = nueva_carpeta / f"Practica_{nueva_practica}"
            nueva_carpeta.mkdir(parents=True, exist_ok=True)
            
            # COMPROBACIÓN MEJORADA DE ARCHIVOS EXISTENTES
            nuevo_path = nueva_carpeta / f"{nombre_base}.pdf"
            
            # Si el archivo destino es el mismo que el origen, no hacer nada
            if nuevo_path == old_path:
                self.status.value = "ℹ️ Sin cambios necesarios"
                return
            
            # Si existe otro archivo con el mismo nombre, buscar sufijo disponible
            contador = 2
            while nuevo_path.exists():
                # Extraer cualquier sufijo numérico existente del nombre base
                match = re.search(r'_(\d+)$', nombre_base)
                if match:
                    # Ya tiene sufijo numérico, incrementarlo
                    numero_actual = int(match.group(1))
                    nombre_sin_sufijo = nombre_base[:match.start()]
                    nuevo_nombre_base = f"{nombre_sin_sufijo}_{numero_actual + contador - 1}"
                else:
                    # No tiene sufijo, añadir uno
                    nuevo_nombre_base = f"{nombre_base}_{contador}"
                
                nuevo_path = nueva_carpeta / f"{nuevo_nombre_base}.pdf"
                contador += 1
                
                # Seguridad: evitar bucle infinito
                if contador > 100:
                    self.status.value = "❌ Error: Demasiados archivos duplicados"
                    return
            
            # Realizar el movimiento de archivo
            try:
                shutil.move(str(old_path), str(nuevo_path))
                
                # Actualizar información en la lista
                current_exam['file_path'] = nuevo_path
                current_exam['carpeta_actual'] = carpeta_destino
                current_exam['practica'] = f"Practica_{nueva_practica}" if carpeta_destino not in ["extraviados", "problemático"] else ""
                current_exam['nombre_archivo'] = nuevo_path.stem
                
                # Mensaje de confirmación
                if carpeta_destino in ["extraviados", "problemático"]:
                    self.status.value = f"✅ Movido a: {carpeta_destino}/{nuevo_path.name}"
                else:
                    self.status.value = f"✅ Movido a: {carpeta_destino}/Practica_{nueva_practica}/{nuevo_path.name}"
                    
                # Si se añadió un sufijo, avisar
                if contador > 2:
                    self.status.value += f" (duplicado evitado con sufijo _{contador-2})"
                    
            except PermissionError:
                self.status.value = "❌ Error: Archivo en uso o sin permisos"
            except FileNotFoundError:
                self.status.value = "❌ Error: Archivo origen no encontrado"
            except Exception as move_error:
                self.status.value = f"❌ Error moviendo archivo: {move_error}"
                
        except Exception as e:
            self.status.value = f"❌ Error aplicando cambios: {e}"

def iniciar_revision_examenes(examenes_dir="../data/examenes_procesados/"):
    print("🚀 Iniciando revisor de exámenes...")
    reviewer = JupyterExamReviewer(examenes_dir)
    return reviewer

In [ ]:
# Para lanzar la interfaz:
#reviewer = iniciar_revision_examenes()

🚀 Iniciando revisor de exámenes...
🔍 Escaneando carpetas de exámenes...
📁 Total de exámenes encontrados: 404


In [ ]:
import zipfile
def crear_backup_examenes(carpeta_examenes="../data/examenes_procesados/", carpeta_backup="../data/"):
    """
    Crea un backup completo de la carpeta de exámenes procesados
    """
    carpeta_examenes = Path(carpeta_examenes)
    carpeta_backup = Path(carpeta_backup)
    
    # Crear nombre del backup con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_backup = f"examenes_procesados_backup_{timestamp}.zip"
    ruta_backup = carpeta_backup / nombre_backup
    
    print(f"🔄 Creando backup de: {carpeta_examenes}")
    print(f"📦 Archivo de backup: {ruta_backup}")
    
    try:
        # Crear el archivo ZIP
        with zipfile.ZipFile(ruta_backup, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Recorrer todos los archivos y carpetas
            for root, dirs, files in os.walk(carpeta_examenes):
                for file in files:
                    file_path = Path(root) / file
                    # Calcular la ruta relativa para mantener la estructura
                    arcname = file_path.relative_to(carpeta_examenes.parent)
                    zipf.write(file_path, arcname)
                    print(f"  ✓ Añadido: {arcname}")
        
        print(f"\n✅ Backup completado exitosamente!")
        print(f"📁 Tamaño del backup: {ruta_backup.stat().st_size / (1024*1024):.2f} MB")
        print(f"💾 Ubicación: {ruta_backup}")
        
        return str(ruta_backup)
        
    except Exception as e:
        print(f"❌ Error creando backup: {e}")
        return None

In [56]:
# Crear el backup
ruta_backup = crear_backup_examenes()

NameError: name 'crear_backup_examenes' is not defined